In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
np.random.seed(132)
from functools import lru_cache

import sys

CODE_PATH = '../code'

sys.path.append(CODE_PATH)
import functions


from scipy.optimize import minimize
from tqdm import tqdm
%pylab inline

def sq(a):
    return np.dot(a, a)

Populating the interactive namespace from numpy and matplotlib


# Load Data

In [3]:
from sklearn.datasets import load_linnerud, load_iris, load_boston, load_breast_cancer, load_wine, load_digits

# dataset = load_linnerud()
# dataset = load_boston()
# dataset = load_boston()
dataset = load_breast_cancer()

df = pd.DataFrame(dataset['data'])
target = dataset['target']
df.head()

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [ ]:
df.shape

In [ ]:
df = pd.read_csv('../data/abalone.csv', header=None)
df = df.drop(0, 1)

In [4]:
# НОРМИРОВКА НА СРЕДНЕЕ, МОЖНО ОПУСТИТЬ ИНОГДА
df = (df - df.mean())/(df.max() - df.min())

In [5]:
df0 = df.copy()
print(df.shape)
df.head()

(569, 30)


,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,0.182815,-0.301307,0.213053,0.146813,0.198968,0.531437,0.495081,0.487976,0.306758,0.335139,...,0.324113,-0.222474,0.385173,0.279792,0.196998,0.399079,0.351207,0.518192,0.335156,0.229268
1,0.304923,-0.051392,0.282848,0.284671,-0.104905,-0.078833,-0.004450,0.105621,0.000193,-0.129057,...,0.310239,-0.060427,0.256680,0.264308,-0.056585,-0.065649,-0.024432,0.245339,-0.029717,0.033282
2,0.263274,0.066295,0.262808,0.232497,0.119524,0.170416,0.254453,0.392549,0.129991,-0.059132,...,0.259723,-0.003924,0.225304,0.203602,0.079452,0.165163,0.142341,0.441216,0.140399,0.023837
3,-0.128132,0.036874,-0.099434,-0.114014,0.416536,0.550761,0.357546,0.279726,0.396657,0.729621,...,-0.048353,0.021929,-0.041791,-0.076898,0.511335,0.593799,0.331239,0.491044,0.736693,0.584115
4,0.291671,-0.167388,0.298051,0.272369,0.035567,0.087292,0.255859,0.275253,-0.001323,-0.083564,...,0.223081,-0.240065,0.223810,0.170669,0.033226,-0.047797,0.102086,0.164583,-0.105806,-0.047001


## Insert NaNs

In [33]:
np.random.seed(180)
num_nan_cols = 15
nan_fraction = 0.1
nan_cols = np.random.random_integers(0, df.shape[1] - 1, num_nan_cols)
# print(df.isnull().mean())
for col in set(nan_cols):
    df.loc[df.sample(int(nan_fraction * len(df))).index, col] = np.nan
# print(df.isnull().mean())
# print(df.isnull().mean())

nan_coords = np.array(np.where(df.isnull().values)).T
print('Num nan places: {}'.format(nan_coords.shape[0]))

Num nan places: 560


/Users/kek/mlenv/lib/python3.6/site-packages/ipykernel_launcher.py:4: DeprecationWarning: This function is deprecated. Please call randint(0, 29 + 1) instead
  after removing the cwd from sys.path.


In [34]:
df1 = df.loc[:, df.isnull().sum() == 0]
df2 = df.fillna(df.mean())
print(df1.shape, df2.shape)
arr_nan = df.values # с пропусками
arr_raw = df0.values # исходные
arr_known = df1.values # суженные до известных признаков
arr_pred = df2.values # текущие предсказанные 

(569, 20) (569, 30)


In [35]:
def iterative_process(update_function, max_iter=20, verbose=True, percentage=False):
    df_result = pd.DataFrame(np.zeros((max_iter, 2)), columns=['mae', 'rmse'])
    for i in range(max_iter):
        update_function()
        if percentage:
            mae = functions.get_mape(arr_raw, arr_pred, nan_coords)
            rmse = functions.get_rmspe(arr_raw, arr_pred, nan_coords)
        else:
            mae = functions.get_mae(arr_raw, arr_pred, nan_coords)
            rmse = functions.get_rmse(arr_raw, arr_pred, nan_coords)
        df_result.loc[i, 'mae'] = mae
        df_result.loc[i, 'rmse'] = rmse
        if verbose:
            print('\tIteration {}  mae {:.6} // rmse {:.6} '.format(i+1, mae, rmse))
    return df_result

In [36]:
mae = functions.get_mae(arr_raw, arr_pred, nan_coords)
rmse = functions.get_rmse(arr_raw, arr_pred, nan_coords)
print(mae, rmse)

0.117532863573 0.152102826747


# Local Approach

In [10]:
from sklearn.neighbors import NearestNeighbors
k_local = 15
teta = 1
nbrs = NearestNeighbors(n_neighbors=k_local, algorithm='ball_tree').fit(arr_pred)

def get_delta_vec(teta):
    deltas = []
    _, heighbours = nbrs.kneighbors(arr_pred)
    for i, j in nan_coords:
        mean_neigh = arr_pred[heighbours[i]].mean(axis=0)[j]
        deltas.append(teta*(mean_neigh - arr_pred[i][j]))
    return np.array(deltas)

def update_local():
    deltas = get_delta_vec(teta)
    nbrs = NearestNeighbors(n_neighbors=k_local, algorithm='ball_tree').fit(arr_pred) # пересчитываем
    for j, (x,y) in enumerate(nan_coords):
        arr_pred[x, y] += deltas[j]

In [11]:
dfr = iterative_process(update_local, max_iter=1000, verbose=True)

	Iteration 1  mae 0.0579182 // rmse 0.0877195 
	Iteration 2  mae 0.0457216 // rmse 0.073915 
	Iteration 3  mae 0.0437857 // rmse 0.0712724 
	Iteration 4  mae 0.0434581 // rmse 0.0706543 
	Iteration 5  mae 0.0432977 // rmse 0.0704729 
	Iteration 6  mae 0.0431949 // rmse 0.0703147 
	Iteration 7  mae 0.0431952 // rmse 0.0702389 
	Iteration 8  mae 0.043207 // rmse 0.0702354 
	Iteration 9  mae 0.0432097 // rmse 0.0702361 
	Iteration 10  mae 0.0432005 // rmse 0.0702297 
	Iteration 11  mae 0.0432 // rmse 0.0702294 
	Iteration 12  mae 0.0432 // rmse 0.0702294 
	Iteration 13  mae 0.0432 // rmse 0.0702294 
	Iteration 14  mae 0.0432 // rmse 0.0702294 
	Iteration 15  mae 0.0432 // rmse 0.0702294 
	Iteration 16  mae 0.0432 // rmse 0.0702294 
	Iteration 17  mae 0.0432 // rmse 0.0702294 
	Iteration 18  mae 0.0432 // rmse 0.0702294 
	Iteration 19  mae 0.0432 // rmse 0.0702294 
	Iteration 20  mae 0.0432 // rmse 0.0702294 
	Iteration 21  mae 0.0432 // rmse 0.0702294 
	Iteration 22  mae 0.0432 // rmse 0.

KeyboardInterrupt: 

In [12]:
k_local/len(df)

0.026362038664323375

# Optimization Approach

In [16]:
n = len(df0)

def get_known_table():
    global arr_nan
    ro_known_table = np.zeros((n, n))
    for i in tqdm(range(n)):
        for j in range(i, n):
            idx_nan_i = np.isnan(arr_nan[i])
            idx_nan_j = np.isnan(arr_nan[j])
            idx = ~idx_nan_i & ~idx_nan_j
            x_i = arr_nan[i, idx]
            x_j = arr_nan[j, idx]
            ro_known_table[i, j] = ro_known_table[j, i] = sq(x_i - x_j)
    return ro_known_table

def get_ro_1_unknown_table():
    global arr_pred
    ro_1_unknown_table = np.zeros((n, n))
    #это ro прибавить там где 1 компонента
    for i in tqdm(range(n)):
        for j in range(i, n):
            idx_nan_i = np.isnan(arr_nan[i])
            idx_nan_j = np.isnan(arr_nan[j])
            idx = (idx_nan_i & ~idx_nan_j) | (~idx_nan_i & idx_nan_j)
            x_i = arr_pred[i, idx]
            x_j = arr_pred[j, idx]
            ro_1_unknown_table[i, j] = ro_1_unknown_table[j, i] = sq(x_i - x_j)
    return ro_1_unknown_table

# по всем признакам
def get_ro_2_unknown_table():
    global arr_pred
    '''
    при первом запуске обычно 0, так как заполняю пропуски ОДНИМ И ТЕМ ЖЕ СРЕДНИМ ЗНАЧЕНИЕМ
    '''
    ro_2_unknown_table = np.zeros((n, n))
    for i in tqdm(range(n)):
        for j in range(i+1, n):
            idx_nan_i = np.isnan(arr_nan[i])
            idx_nan_j = np.isnan(arr_nan[j])
            idx = (idx_nan_i & idx_nan_j)
            x_i = arr_pred[i, idx]
            x_j = arr_pred[j, idx]
            ro_2_unknown_table[i, j] = ro_2_unknown_table[j, i] = sq(x_i - x_j)
    return ro_2_unknown_table

In [17]:
ro_known_table = get_known_table()
ro_1_unknown_table = get_ro_1_unknown_table()
ro_2_unknown_table = get_ro_2_unknown_table() # здесь какого-то хера одни нули!!!

100%|██████████| 569/569 [00:03<00:00, 169.47it/s]


In [18]:
def F_functional(k_arr, ftype='f1'):
    global ro_known_table, ro_1_unknown_table, ro_2_unknown_table
    f = 0
    for i in tqdm(range(n)):
        for j in range(i+1, n):
            ro1 = ro_known_table[i, j] # известно оба 
            ro2 = ro_1_unknown_table[i, j] # известен 1 
            ro3 = ro_2_unknown_table[i, j] # не известно ниче
            
            r1 = np.sqrt(ro1 + ro2 + ro3) #по всем признакам(р)
            if ftype == 'f1':
                r2 = np.sqrt(ro1) # по известным признакам(р+)
            else:
                r2 = np.sqrt(ro1 + ro2)
            f += (r1 - k_arr[i, j]*r2)**2
    return f

In [19]:
### ПРОВЕРИТЬ ГРАДИЕНТ НА 0 ДЛЯ Ф2

def grad_component(i, j, k_arr, ftype='f1'):
    global ro_known_table, ro_1_unknown_table, ro_2_unknown_table
    s = 0
    for i1 in range(n):
        if i1 == i:
            continue
        ro1 = ro_known_table[i, i1]
        ro2 = ro_1_unknown_table[i, i1]
        ro3 = ro_2_unknown_table[i, i1]
        
        r1 = np.sqrt(ro1 + ro2 + ro3) #по всем признакам(р)
        if r1 <= 0:
            continue
        if ftype == 'f2':
            r2 = np.sqrt(ro1 + ro2) # по известным признакам(р+)
            delta_x1 = (arr_pred[i, j] - arr_pred[i1, j])
            if not np.isnan(arr_nan[i1, j]):
                delta_x2 = delta_x1
            else:
                delta_x2 = 0
            if r2 == 0:
                delta_x2 = k_arr[i1, i]
            else:
                delta_x2 = k_arr[i1, i]*delta_x2/r2
            delta_s = 2*(r1 - k_arr[i1, i]*r2)*(delta_x1/r1 - delta_x2)
            s += delta_s
        else:
            r2 = np.sqrt(ro1) # по известным признакам(р+)
            s += 2 * (1 - k_arr[i1, i]*r2/r1) * (arr_pred[i, j] - arr_pred[i1, j])
    return s

def get_full_grad(nan_coords, k_arr, ftype='f1'):
    grad = []
    for i, j in nan_coords:
        grad.append(grad_component(i, j, k_arr, ftype))
    return np.array(grad)

In [20]:
k_arr = np.ones((n, n))
ftype = 'f1'
alpha = 0.005
alpha = 0.01
c = F_functional(k_arr, ftype)

100%|██████████| 569/569 [00:01<00:00, 370.32it/s]


In [ ]:
c = F_functional(k_arr, ftype)

In [ ]:
# Для F2 нужно делать не единички, а нормировать
# k_arr[i, j] = n - n2 / n (то есть все признаки без двух пропусков/на все признаки)
k_arr = np.ones((n, n))
alpha = 1.1
for i in tqdm(range(n)):
    for j in range(i+1, n):
        idx_nan_i = np.isnan(arr_nan[i])
        idx_nan_j = np.isnan(arr_nan[j])
        idx = (idx_nan_i & idx_nan_j)
        k_arr[i, j] = k_arr[j, i] = (n - idx.sum()) / n
ftype = 'f2'
c = F_functional(k_arr, ftype)

In [21]:
# проверка изменений в зависимости от альфы
arr_pred_raw = arr_pred.copy()
grad = get_full_grad(nan_coords, k_arr, ftype)

def get_f_grad(arr_pred_raw, alpha, grad):
    global ro_1_unknown_table, ro_2_unknown_table
    for j, (x,y) in enumerate(nan_coords):
        arr_pred[x, y] = arr_pred_raw[x, y] - alpha*grad[j]
    ro_1_unknown_table = get_ro_1_unknown_table()
    ro_2_unknown_table = get_ro_2_unknown_table()
    c = F_functional(k_arr, ftype)
    return c

In [22]:
c_array = []

ro1t = []
ro2t = []
def update_opt():
    global ro_1_unknown_table, ro_2_unknown_table
    grad = get_full_grad(nan_coords, k_arr, ftype)
    for j, (x,y) in enumerate(nan_coords):
        arr_pred[x, y] = arr_pred[x, y] - alpha*grad[j]
        
    ro_1_unknown_table = get_ro_1_unknown_table()
    ro_2_unknown_table = get_ro_2_unknown_table()
    c = F_functional(k_arr, ftype)
    print('C = {}'.format(c))
    c_array.append(c)
    ro1t.append(ro_1_unknown_table)
    ro2t.append(ro_2_unknown_table)

In [23]:
max_iter = 30

dfr = iterative_process(update_opt, max_iter=max_iter, verbose=True)

100%|██████████| 569/569 [00:01<00:00, 304.01it/s]


C = 188.89417211345412
	Iteration 1  mae 0.0999151 // rmse 0.130778 


100%|██████████| 569/569 [00:02<00:00, 207.82it/s]


C = 186.65581244925562
	Iteration 2  mae 0.0990311 // rmse 0.128556 


100%|██████████| 569/569 [00:01<00:00, 315.94it/s]


...

100%|██████████| 569/569 [00:01<00:00, 293.21it/s]


C = 185.88575067487426
	Iteration 8  mae 0.0973295 // rmse 0.125557 


100%|██████████| 569/569 [00:01<00:00, 292.29it/s]


C = 185.88273838795675
	Iteration 9  mae 0.0972751 // rmse 0.125484 


KeyboardInterrupt: 

In [ ]:
print(c_array, ftype)

for i in range(7):
    print((ro2t[i] - ro2t[i+1]).max())

# SNE Approach

In [27]:
n = len(df0)
# таблица старых значений
Exp1_table = np.zeros((n, n))
for i in tqdm(range(n)):
    for j in range(n):
        Exp1_table[i][j] = np.exp(-sq(arr_known[i] - arr_known[j]))
#         Exp1_table[i][j] = np.exp(-norm(arr_known[i] - arr_known[j])**2)
        
P1_table = np.zeros((n, n))
for i in tqdm(range(n)):
    for j in range(n):
        a = Exp1_table[j][i]
        b = Exp1_table[i].sum() - 1 # 1 = Exp1_table[i][i]
        P1_table[j][i] = a / b

100%|██████████| 569/569 [00:03<00:00, 150.76it/s]


In [28]:
def get_exp2_table():
    Exp2_table = np.zeros((n, n))
    for i in tqdm(range(n)):
        for j in range(n):
            Exp2_table[i][j] = np.exp(-sq(arr_pred[i] - arr_pred[j]))
#             Exp2_table[i][j] = np.exp(-norm(arr_pred[i] - arr_pred[j])**2)
    return Exp2_table

def get_p2_table():
    Exp2_table = get_exp2_table()
    P2_table = np.zeros((n, n))
    for i in tqdm(range(n)):
        for j in range(n):
            a = Exp2_table[j][i]
            b = Exp2_table[i].sum() - 1 # 1 = Exp1_table[i][i]
            P2_table[j][i] = a / b
    return P2_table
P2_table = get_p2_table()

100%|██████████| 569/569 [00:03<00:00, 151.29it/s]


In [29]:
x = 1
def f1():
    global x
    return x*2
def f2():
    y = f1()
    return y

for i in range(10):
    print(f2())
    x += 1

2
4
6
8
10
12
14
16
18
20


In [30]:
def KL(P2_table):
    global P1_table
#     global P1_table, P2_table
    s = 0
    for i in range(n):
        for j in range(n):
#             s += P1_table[j][i] * np.log(P1_table[j][i] / P2_table[j][i])
            s += P1_table[j][i] * np.log(P1_table[j][i]) - P1_table[j][i] * np.log(P2_table[j][i])
    return s

def get_grad_sne(i1, i2):
    global P1_table, P2_table
    def get_i_part(i):
        d = 2*(arr_pred[i1][i2] - arr_pred[i][i2])
        s = (P1_table[i1][i]+P1_table[i][i1]) - \
            P2_table[i1][i]*(1+P1_table[i][i]) - \
            P2_table[i][i1]*(1+P1_table[i1][i1])
        return s*d
    return sum(get_i_part(i) for i in range(n) if i!=i1)

def get_full_grad_sne(nan_coords):
    n_gaps = len(nan_coords)
    grad = np.zeros(n_gaps)
    for i in range(n_gaps):
        i1, i2 = nan_coords[i]
        grad[i] = get_grad_sne(i1, i2)
    return grad

In [31]:
momentum = True
alpha_momentum = 0.5
c_array = []
p_array = []
g_array = []

y_momentum = np.zeros(len(nan_coords))
for j, (x,y) in enumerate(nan_coords):
    y_momentum[j] = arr_pred[x, y]
y_array = [y_momentum]

def update_sne():
    grad = get_full_grad_sne(nan_coords)

    for j, (x,y) in enumerate(nan_coords):
        arr_pred[x, y] = arr_pred[x, y] - alpha*grad[j]
    # если есть еще предыдущие, то делаем шаг моментум
    if momentum and len(y_array) >= 2:
        if len(y_array) >= 2:
            y_diff = y_array[-1] - y_array[-2]
            for j, (x,y) in enumerate(nan_coords):
                arr_pred[x, y] += alpha_momentum*y_diff[j]
        y_momentum = np.zeros(len(nan_coords))
        for j, (x,y) in enumerate(nan_coords):
            y_momentum[j] = arr_pred[x, y]
        y_array.append(y_momentum)
    
    P2_table = get_p2_table()
    c = KL(P2_table)
    print('C = {}'.format(c))
    c_array.append(c)
    p_array.append(P2_table)
    g_array.append(grad)

In [ ]:
def KL_local(P2_table):
    global P1_table
    s = 0
    for i in range(n):
        for j in range(n):
            s += P1_table[j][i] * np.log(P1_table[j][i]) - P1_table[j][i] * np.log(P2_table[j][i])
    return s
for i in range(7):
    print(KL_local(p_array[i]))

In [32]:
alpha = 0.05
dfr = iterative_process(update_sne, max_iter=150, verbose=True)

100%|██████████| 569/569 [00:03<00:00, 163.41it/s]


C = 15.09692400276303
	Iteration 1  mae 0.119186 // rmse 0.153918 


100%|██████████| 569/569 [00:03<00:00, 170.46it/s]


C = 15.035986995639504
	Iteration 2  mae 0.120838 // rmse 0.155738 


100%|██████████| 569/569 [00:03<00:00, 162.74it/s]


C = 14.976445091719306
	Iteration 3  mae 0.122493 // rmse 0.157564 


100%|██████████| 569/569 [00:03<00:00, 167.33it/s]


C = 14.918281227520248
	Iteration 4  mae 0.124147 // rmse 0.159394 


 97%|█████████▋| 554/569 [00:05<00:00, 102.33it/s]


KeyboardInterrupt: 

In [ ]:
s = 0
for i in range(n):
    for j in range(n):
        s += P1_table[j][i] * np.log(P1_table[j][i] / P2_table[j][i])
    print(s)
return s

# AVO APPROACH
Единственный подход, который не содержит оптимизайионный подход

In [37]:
def get_epsilons(data):
    def get_e(a):
        return np.abs(a - a[:, None]).mean()
    return np.array([get_e(feat) for feat in data.T])

In [38]:
def get_gamma(data, x, k, epsilons=None):
    if epsilons is not None:
        Gamma = 0
        for x_i in data:
            d = (np.abs(x_i - x) < epsilons).sum()
            Gamma += functions.Cnk(d, k)
        return Gamma
    # расстояния между парами объектов   
    n = data.shape[0]
    n_feat = data.shape[1]
    maps = np.zeros(data.T.shape)
    for i in range(n_feat):
        maps[i] = data.T[i] <= x[i] 
    maps = maps.T
    Gamma = 0
    for i in tqdm(range(n)):
        for j in range(i, n):
            d = (maps[i] + maps[j] == 1).sum()
            Gamma += functions.Cnk(d, k)
    return Gamma*2/(n*(n-1))

In [39]:
def predict(data, y, x, k, epsilons=None, scores=False):
    # WTF ZERO
    n_class = max(y) + 1
    g_classes = np.zeros(n_class)
    y = y.values 
    for i in range(n_class):
        c_data = data[y == i]
        g_classes[i] = get_gamma(c_data, x, k, epsilons)
    if scores:
        return g_classes
    return np.argmax(g_classes)

def predict_vect(X_train, y_train, X_test, k, epsilons=None, scores=False):
    return np.array([predict(X_train, y_train, x, k, epsilons, scores) for x in X_test])

In [40]:
# итеративный процесс
def solve_avo(df, x, y):
    x_mask = df.iloc[:, y].isnull() 
    x_mask = x_mask[~x_mask].index # маска по объектам для трейна
    y_mask = df.isnull().sum() > 0 
    y_mask = y_mask[~y_mask].index # маска по по признакам для трейна/теста
    
    X_train = df.iloc[x_mask, y_mask]
    X_test = df.iloc[x, y_mask].values.reshape(1, -1)

    y_train_raw = df.iloc[x_mask, y]
    y_train = y_train_raw.rank(method='dense').astype(int) - 1
    
    eps = get_epsilons(X_train.values)
    gammas_ki = predict(X_train.values, y_train, X_test, 3, eps, True)
    sizes_ki = y_train.value_counts().sort_index().values
    
    full_range = np.array(range(sizes_ki.size))
    sort_index = solve_avo_gap(gammas_ki, sizes_ki, full_range)
    return y_train_raw.sort_values().iloc[sort_index]
    
def solve_avo_gap(gammas_ki, sizes_ki, full_range):
#     print(full_range)
    if len(full_range) == 1:
        return full_range[0]
    left_range = full_range[:full_range.size//2]
    right_range = full_range[full_range.size//2:]
    left_score = get_gamma_range(gammas_ki, sizes_ki, left_range)
    right_score = get_gamma_range(gammas_ki, sizes_ki, right_range)
    if left_score < right_score:
        return solve_avo_gap(gammas_ki, sizes_ki, right_range)
    else:
        return solve_avo_gap(gammas_ki, sizes_ki, left_range)
        
def get_gamma_range(gammas_ki, sizes_ki, y_range):
    sum_gammas = gammas_ki[y_range].sum()
    sum_sizes = sizes_ki[y_range].sum()
    if sum_sizes == 0:
        return 0
    return sum_gammas/sum_sizes

In [ ]:
# по всем пропускам из таблицы запускаем и проходим
for x, y in tqdm(nan_coords):
    arr_pred[x, y] = solve_avo(df, x, y)

 45%|████▌     | 252/560 [00:15<00:19, 15.78it/s]

In [ ]:
# for x, y in tqdm(nan_coords):
#     print(arr_pred[x, y])

In [ ]:
mae = functions.get_mae(arr_raw, arr_pred, nan_coords)
rmse = functions.get_rmse(arr_raw, arr_pred, nan_coords)
print(mae, rmse)

In [ ]:

7.62472820045 14.1056553537